<a href="https://colab.research.google.com/github/vanditagoyal1997/depression-analysis/blob/vandita/Audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
import keras
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile as wav
from scipy import signal
# from python_speech_features import mfcc, logfbank
from scipy.fftpack import fft,fftfreq
import os
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df=pd.read_csv('train_split_Depression_AVEC2017.csv')
os.getcwd()
df
y=df['PHQ8_Binary']
id=df['Participant_ID']
id.tolist()
df

FileNotFoundError: ignored

In [0]:
os.chdir('E:/final year project/Spectrogram')
new_list=[]
f1=os.listdir()
for i in f1:
    print(int(i[:3]))
    if int(i[:3]) in id:
        new_list.append(i[:3])

FileNotFoundError: ignored

In [0]:
new_list

[]

In [0]:
os.chdir('E:/final year project')
os.chdir('E:/final year project/Audio')
f=os.listdir()


# generating spectogram images from audio data

In [0]:
a=os.listdir()

for i in a:
    w=wave.open(i,'rb')
    frames = w.readframes(-1)
    sound_info = pylab.fromstring(frames, 'Int16')
    frame_rate = w.getframerate()
    w.close()
    pylab.figure(num=None, figsize=(513, 125))
    pylab.subplot(111)
    pylab.title('spectrogram of the audio file')
    pylab.specgram(sound_info, Fs=frame_rate)
    pylab.savefig('E:/final year project/Spectrogram/'+i[0:-3]+'spec.png')

# cnn model for the spectogram images

In [0]:
num_classes=2
y=[0,0,0]
epochs=1
y=to_categorical(y,num_classes=2)

In [0]:
model = Sequential()

In [0]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/final_year_proj/spectogram')
a=os.listdir()
arr=[]
for i in a:
    t=load_img(i)
    x=img_to_array(t)
#     x = x.reshape((1,) + x.shape)
    arr.append(x)
# arr = to_categorical(arr, num_classes=2)
# np.array(arr,dtype=np.float32)

In [0]:
os.chdir('E:/final year project/Spec_test')
a=os.listdir()
y_test=[0,1,1]
y_test=to_categorical(y_test,num_classes=2)
len(a)
x_test=[]
for i in a:
    t=load_img(i)
    x=img_to_array(t)
#     x = x.reshape((1,) + x.shape)
    x_test.append(x)
# np.array(arr,dtype=np.float32)


In [0]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# y_train.shape
x.shape
# len(x_test)

(1080, 1080, 3)

In [0]:
# arr[0]

In [0]:
model.add(Conv2D(3, kernel_size=3, activation='relu', input_shape=(1080,1080,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(Conv2D(3, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1078, 1078, 3)     84        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 539, 539, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 537, 537, 3)       84        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 268, 268, 3)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 215472)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               110322176 
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
__________

In [0]:
model.fit(np.array(arr), np.array(y),epochs=epochs,verbose=1,validation_data=(np.array(x_test), np.array(y_test)))

Instructions for updating:
Use tf.cast instead.
Train on 3 samples, validate on 3 samples
Epoch 1/1
3/3 [==============================] - 38s 13s/step - loss: 10.7454 - acc: 0.3333 - val_loss: 10.4698 - val_acc: 0.3333
